In [3]:
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI

In [4]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=False
)
agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power??"
)

Starting chain trace with inputs: {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power??"}, serialized: {'name': 'AgentExecutor'}, extra: {}
Starting chain trace with inputs: {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power??", 'agent_scratchpad': '', 'stop': ['\nObservation: ']}, serialized: {'name': 'LLMChain'}, extra: {}
Starting LLM trace with prompts: ["Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Acti

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."